In [ ]:
###### setup #######

from jetbot import ObjectDetector
from jetbot import Camera
from jetbot import Robot
from jetbot import bgr8_to_jpeg
import ipywidgets.widgets as widgets
import cv2
import numpy as np
import time

model = ObjectDetector('ssd_mobilenet_v2_coco.engine')
camera = Camera.instance(width=300, height=300, fps=5)
robot = Robot()

In [ ]:
###### definitions #######

target = 1 # arbitrary final target (human)
obstacle = 3 # arbitrary obstacle (apple)

objectDictionary = {1:('human', 200), 3:('apple', 30)}

In [ ]:
####### other utilities #########

imageFeed = widgets.Image(format='jpeg', width=300, height=300)
speedSlider = widgets.FloatSlider(value=0.3, min=0.0, max=1.0, description='speed')

width = int(imageFeed.width)
height = int(imageFeed.height)

# display(widgets.VBox([widgets.HBox([imageFeed]), speedSlider]))
    # moved down during testing, bring back up or just delete later on

In [ ]:
###### non essemtial helpers, not finalized ########


def incrementalLeft():
    robot.left(speedSlider.value) # arbitrary small value
    time.sleep(0.15) # can change, might want to go higher
    robot.stop()
    
def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def center(detection):
    # move robot forward and steer proportional target's x-distance from center
    center = detection_center(detection)
    incrementalLeft() # substitutes until i write the actual center code
    incrementalLeft()
    # print("x center offset: ", center[0])

In [ ]:
display(widgets.VBox([widgets.HBox([imageFeed]), speedSlider]))
    # just here to see easier when test; remove later on

###### essential helpers + main ########

def notInVision():
    inVision = False
    while(not inVision):  
        
        incrementalLeft()
        time.sleep(0.5)
            
        image = camera.value
        detections = model(image)

        for detection in detections[0]:        
            bbox = detection['bbox']
            label = detection['label']
            if label == target:
                cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 0, 255), 2)
                print('detected after turning')
                center(detection)
                inVision = True
        
        imageFeed.value = bgr8_to_jpeg(image)

def lockOn(image):
    print('starting lock')
    isFound = False
    detections = model(image)
    
    for detection in detections[0]:        
        label = detection['label']
        if label == 1:
            print('in view initially; detected')
            isFound = True
            center(detection)
    
    if not isFound:
        notInVision()
    else:
        imageFeed.value = bgr8_to_jpeg(image)

    # post centering image, no bboxs, delay to see effect
    time.sleep(1)
    imageFeed.value = bgr8_to_jpeg(camera.value)
        
    print('end of lock and center, waiting 3 second(s)')
    time.sleep(3)

def run(change):
    image = change['new']
    
    detections = model(image)
    imageFeed.value = bgr8_to_jpeg(image)
    
    for detection in detections[0]:
        bbox = detection['bbox']
        label = detection['label']
        
        if label == target:
            cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 0, 255), 2)
            cv2.putText(image, objectDictionary[label][0] + ":", (int(width * bbox[0]), int(height * bbox[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255),2)
        
        elif label == obstacle:
            cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 0, 255), 2)
            cv2.putText(image, objectDictionary[label][0] + ":", (int(width * bbox[0]), int(height * bbox[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255),2)
        
    lockOn(image)
    
    # additional methds, although the lock on method will probably just be added to original file

In [ ]:
###### running ########

for i in range(0,10): # could be while not isReached
    run({'new': camera.value})
    
print('end of the loop (maybe the target was reached?)')

# was having issues with the following
# the camera would use too much of the memory and would not update, took the easy way out 
#            and looped "run({'new': camera.value})" 
# https://github.com/NVIDIA-AI-IOT/jetbot/issues/120
# https://github.com/NVIDIA-AI-IOT/jetbot/issues/63

# camera.unobserve_all()
# camera.observe(run, names='value')

In [ ]:
####### turning off #########

camera.unobserve_all()
robot.stop()